# Process Cash and Dividends

In [73]:
# Derive data from trades.csv
import pandas as pd
import numpy as np

trades_df = pd.read_csv('data/core/input/trades.csv')
trades_df['Date'] = pd.to_datetime(trades_df['Date'])
trades_df.set_index('Date', inplace=True)

# Build a dictionary of ticker to currency
ticker_to_currency = {}
for index, row in trades_df.iterrows():
    ticker = row['Ticker']
    currency = row['Currency']
    ticker_to_currency[ticker] = currency
ticker_to_currency

{'EA': 'USD',
 'ISRG': 'USD',
 'MA': 'USD',
 'TEX': 'USD',
 'AAPL': 'USD',
 'AGG': 'USD',
 'APO': 'USD',
 'CEG': 'USD',
 'HBM.TO': 'CAD',
 'L.TO': 'CAD',
 'SPY': 'USD',
 'XBB.TO': 'CAD',
 'XIU.TO': 'CAD',
 'AMSF': 'USD',
 'GSL': 'USD',
 'SPSB': 'USD',
 'VEEV': 'USD',
 'CSH-UN.TO': 'CAD',
 'WFG.TO': 'CAD',
 'ACO-X.TO': 'CAD',
 'AER': 'USD',
 'AMAT': 'USD',
 'BLBD': 'USD',
 'CG': 'USD',
 'DOLE': 'USD',
 'MP': 'USD',
 'TMUS': 'USD',
 'WSC': 'USD',
 'SCHP': 'USD',
 'TLT': 'USD',
 'FFH.TO': 'CAD',
 'GOOG': 'USD',
 'RY.TO': 'CAD',
 'CGY.TO': 'CAD',
 'IAU': 'USD',
 'LUNR': 'USD'}

In [74]:
# Join trades with exchange rates (on trade date)
exchange_rates = pd.read_csv('data/core/output/exchange_rates.csv')
exchange_rates['Date'] = pd.to_datetime(exchange_rates['Date'])
exchange_rates.set_index('Date', inplace=True)
df = pd.merge(trades_df, exchange_rates, left_on='Date', right_on='Date', how='left')
df.head()

,Ticker,Currency,Quantity,Price,CAD,USD
Date,,,,,,
2022-05-05,EA,USD,16.0,121.57,1.0,1.27372
2022-05-05,ISRG,USD,8.0,238.04,1.0,1.27372
2022-05-05,MA,USD,5.0,358.41,1.0,1.27372
2022-05-05,TEX,USD,56.0,34.66,1.0,1.27372
2023-04-27,AAPL,USD,4.0,165.39,1.0,1.36307


In [75]:
# accumulate the cash flows for USD and CAD separately

CAD = 101644.99 # CAD
USD = 0.0
LATEST_EXCHANGE_RATE = 1.4045

# process buys
for index, row in df.iterrows():
    c = row['Currency']
    q = row['Quantity']
    p = row['Price']
    er = row['USD']

    # skip sells
    if q < 0:
        continue

    if c == 'CAD':
        CAD -= q * p
    else:
        # if we don't have enough USD, convert CAD to USD then make purchase
        if USD < q * p:
            CAD -= q * p * er
        else:
            USD -= q * p

print(f'CAD after only buys: {CAD}')
print(f'USD after only buys (should be 0): {USD}')

# process sells
for index, row in df.iterrows():
    c = row['Currency']
    q = row['Quantity']
    p = row['Price']
    er = row['USD']

    # skip buys
    if q > 0:
        continue

    if c == 'CAD':
        CAD += -q * p
    else:
        USD += -q * p


print(f"CAD after buys and sells: {CAD}")
print(f"USD after buys and sells: {USD}")
print(f'total: {CAD + USD * df["USD"].iloc[-1]}')
print(f'total using todays exchange rate: {CAD + USD * LATEST_EXCHANGE_RATE}')



CAD after only buys: -124079.32808119993
USD after only buys (should be 0): 0.0
CAD after buys and sells: -82374.98808119993
USD after buys and sells: 88528.0885
total: 41962.71289246576
total using todays exchange rate: 41962.712217050066


# Let's try processing trades with conversions when balance is too low

In [76]:
# accumulate the cash flows for USD and CAD separately

CAD = 101644.99 # CAD
USD = 0.0
LATEST_EXCHANGE_RATE = 1.4045
avg_er_sum = 0.0
avg_er_count = 0

# process buys
for index, row in df.iterrows():
    c = row['Currency']
    q = row['Quantity']
    p = row['Price']
    er = row['USD']

    print(f'processing trade {index} q {q} p {p} er {er}')

    if q < 0:
        if c == 'CAD':
            CAD += -q * p
        else:
            USD += -q * p
    elif q > 0:
        if c == 'CAD':
            if CAD < q * p:
                # need to convert USD to CAD
                cad_needed = q * p - CAD
                usd_to_convert = cad_needed / er
                USD -= usd_to_convert
                CAD += cad_needed

                avg_er_sum += er
                avg_er_count += 1

                # make trade with CAD
                CAD -= q * p
            else:
                CAD -= q * p
        elif c == 'USD':
            # if we don't have enough USD, convert CAD to USD then make purchase
            if USD < q * p:
                # need to convert CAD to USD
                usd_needed = q * p - USD
                cad_to_convert = usd_needed * er
                CAD -= cad_to_convert
                USD += usd_needed

                avg_er_sum += er
                avg_er_count += 1

                # make trade with USD
                USD -= q * p
            else:
                USD -= q * p
        else:
            raise ValueError(f"Unsupported currency: {c}")
    
    print(f'CAD: {CAD}, USD: {USD} after trade')

# print(f'CAD after only buys: {CAD}')
# print(f'USD after only buys (should be 0): {USD}')

# process sells
# for index, row in df.iterrows():
#     c = row['Currency']
#     q = row['Quantity']
#     p = row['Price']
#     er = row['USD']

#     # skip buys
#     if q > 0:
#         continue

    


print(f"CAD after buys and sells: {CAD}")
print(f"USD after buys and sells: {USD}")
print(f'total: {CAD + USD * df["USD"].iloc[-1]} using exchange rate: {df["USD"].iloc[-1]}')
print(f'avg er used: {avg_er_sum / avg_er_count}')
print(f'total using avg er: {CAD + USD * (avg_er_sum / avg_er_count)}')
print(f'time had to convert for a trade: {avg_er_count}')
print(len(trades_df))



processing trade 2022-05-05 00:00:00 q 16.0 p 121.57 er 1.2737200260162354
CAD: 99167.45170299531, USD: 0.0 after trade
processing trade 2022-05-05 00:00:00 q 8.0 p 238.04 er 1.2737200260162354
CAD: 96741.88118305207, USD: 0.0 after trade
processing trade 2022-05-05 00:00:00 q 5.0 p 358.41 er 1.2737200260162354
CAD: 94459.31121042967, USD: 0.0 after trade
processing trade 2022-05-05 00:00:00 q 56.0 p 34.66 er 1.2737200260162354
CAD: 91987.0715887332, USD: 0.0 after trade
processing trade 2023-04-27 00:00:00 q 4.0 p 165.39 er 1.363070011138916
CAD: 91085.31899216415, USD: 0.0 after trade
processing trade 2023-04-27 00:00:00 q 155.0 p 97.83 er 1.363070011138916
CAD: 70416.20241775752, USD: 0.0 after trade
processing trade 2023-04-27 00:00:00 q 30.0 p 62.36 er 1.363070011138916
CAD: 67866.17104091884, USD: 0.0 after trade
processing trade 2023-04-27 00:00:00 q 25.0 p 76.42 er 1.363070011138916
CAD: 65262.02578463794, USD: 0.0 after trade
processing trade 2023-04-27 00:00:00 q 225.0 p 6.68

# Process Dividends

In [77]:
# Process dividends
dividends = pd.read_csv('data/core/output/dividend_income.csv')
dividends['Date'] = pd.to_datetime(dividends['Date'])
dividends.set_index('Date', inplace=True)

# accumulate the dividend income for USD and CAD separately
CAD_dividends = 0.0
USD_dividends = 0.0

assert len(ticker_to_currency) == len(dividends.columns)

for index, row in dividends.iterrows():
    for ticker in dividends.columns:
        print(ticker)
        currency = ticker_to_currency[ticker]
        if currency == 'CAD':
            CAD_dividends += row[ticker]
        else:
            USD_dividends += row[ticker]

print(f'CAD total dividends: {CAD_dividends}')
print(f'USD total dividends: {USD_dividends}')
print(f'total dividends in CAD: {CAD_dividends + USD_dividends * LATEST_EXCHANGE_RATE}')



AAPL
ACO-X.TO
AER
AGG
AMAT
AMSF
APO
BLBD
CEG
CG
CGY.TO
CSH-UN.TO
DOLE
EA
FFH.TO
GOOG
GSL
HBM.TO
IAU
ISRG
L.TO
LUNR
MA
MP
RY.TO
SCHP
SPSB
SPY
TEX
TLT
TMUS
VEEV
WFG.TO
WSC
XBB.TO
XIU.TO
AAPL
ACO-X.TO
AER
AGG
AMAT
AMSF
APO
BLBD
CEG
CG
CGY.TO
CSH-UN.TO
DOLE
EA
FFH.TO
GOOG
GSL
HBM.TO
IAU
ISRG
L.TO
LUNR
MA
MP
RY.TO
SCHP
SPSB
SPY
TEX
TLT
TMUS
VEEV
WFG.TO
WSC
XBB.TO
XIU.TO
AAPL
ACO-X.TO
AER
AGG
AMAT
AMSF
APO
BLBD
CEG
CG
CGY.TO
CSH-UN.TO
DOLE
EA
FFH.TO
GOOG
GSL
HBM.TO
IAU
ISRG
L.TO
LUNR
MA
MP
RY.TO
SCHP
SPSB
SPY
TEX
TLT
TMUS
VEEV
WFG.TO
WSC
XBB.TO
XIU.TO
AAPL
ACO-X.TO
AER
AGG
AMAT
AMSF
APO
BLBD
CEG
CG
CGY.TO
CSH-UN.TO
DOLE
EA
FFH.TO
GOOG
GSL
HBM.TO
IAU
ISRG
L.TO
LUNR
MA
MP
RY.TO
SCHP
SPSB
SPY
TEX
TLT
TMUS
VEEV
WFG.TO
WSC
XBB.TO
XIU.TO
AAPL
ACO-X.TO
AER
AGG
AMAT
AMSF
APO
BLBD
CEG
CG
CGY.TO
CSH-UN.TO
DOLE
EA
FFH.TO
GOOG
GSL
HBM.TO
IAU
ISRG
L.TO
LUNR
MA
MP
RY.TO
SCHP
SPSB
SPY
TEX
TLT
TMUS
VEEV
WFG.TO
WSC
XBB.TO
XIU.TO
AAPL
ACO-X.TO
AER
AGG
AMAT
AMSF
APO
BLBD
CEG
CG
CGY.TO
CSH-UN.TO
DOLE
EA
FFH.TO


In [78]:
# Sum up the total cash flows
CAD_total = CAD + CAD_dividends
USD_total = USD + USD_dividends

print(f"CAD before dividends: {CAD}")
print(f"CAD dividends: {CAD_dividends}")
print(f'CAD total: {CAD_total}')
print(f"USD before dividends: {USD}")
print(f"USD dividends: {USD_dividends}")
print(f'USD total: {USD_total}')
print(f'total in CAD: {CAD_total + USD_total * LATEST_EXCHANGE_RATE}')


CAD before dividends: 0.0
CAD dividends: 2803.457
CAD total: 2803.457
USD before dividends: 28888.796024660558
USD dividends: 3598.234
USD total: 32487.03002466056
total in CAD: 48431.49066963576


# Process currency conversion as well

In [79]:
# Date,Currency_From,Currency_To,Amount,Rate
# 2025-10-14,USD,CAD,2166.75,1.3821

USD_total -= 2166.75
CAD_total += 2166.75 * 1.3821

print(f'CAD total: {CAD_total}')
print(f'USD total: {USD_total}')
print(f'total in CAD: {CAD_total + USD_total * LATEST_EXCHANGE_RATE}')



CAD total: 5798.122175
USD total: 30320.28002466056
total in CAD: 48382.95546963575


# Process Holdings

In [80]:
# read daily holdings
holdings = pd.read_csv('data/core/output/daily_holdings.csv')
holdings['Date'] = pd.to_datetime(holdings['Date'])
holdings.set_index('Date', inplace=True)

# merge with exchange rates
holdings = pd.merge(holdings, exchange_rates, left_on='Date', right_on='Date', how='left')
holdings.head()

,AAPL,ACO-X.TO,AER,AGG,AMAT,AMSF,APO,BLBD,CEG,CG,...,TEX,TLT,TMUS,VEEV,WFG.TO,WSC,XBB.TO,XIU.TO,CAD,USD
Date,,,,,,,,,,,,,,,,,,,,,
2022-05-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.28530
2022-05-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.28740
2022-05-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.28338
2022-05-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,56.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.27372
2022-05-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,56.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.28270


In [81]:
# get price data
prices = pd.read_csv('data/core/output/prices.csv')
prices['Date'] = pd.to_datetime(prices['Date'])
prices.set_index('Date', inplace=True)
prices.head()

,AAPL,ACO-X.TO,AER,AGG,AMAT,AMSF,APO,BLBD,CEG,CG,...,SPSB,SPY,TEX,TLT,TMUS,VEEV,WFG.TO,WSC,XBB.TO,XIU.TO
Date,,,,,,,,,,,,,,,,,,,,,
2022-05-02,157.960007,44.660000,46.680000,102.129997,112.970001,47.070000,51.919998,16.150000,58.200001,37.049999,...,29.840000,414.480011,34.689999,117.180000,126.400002,187.589996,114.440002,34.840000,28.100000,31.780001
2022-05-03,159.479996,44.599998,47.450001,102.360001,113.800003,46.619999,51.730000,15.980000,59.700001,38.610001,...,29.830000,416.380005,34.439999,117.970001,126.250000,183.699997,117.169998,35.910000,28.070000,32.099998
2022-05-04,166.020004,45.389999,48.580002,103.019997,118.809998,48.000000,53.660000,16.549999,61.810001,40.540001,...,29.930000,429.059998,36.610001,118.620003,132.369995,188.190002,119.050003,36.630001,28.250000,32.480000
2022-05-05,156.770004,45.630001,47.360001,102.000000,113.470001,47.470001,53.369999,15.720000,60.160000,38.389999,...,29.870001,413.809998,34.869999,115.370003,129.559998,178.449997,118.680000,34.660000,27.969999,31.770000
2022-05-06,157.279999,46.279999,46.259998,101.570000,112.500000,48.240002,52.900002,15.480000,60.270000,37.689999,...,29.860001,411.339996,34.320000,113.669998,126.800003,174.860001,117.230003,34.259998,27.809999,31.680000


In [82]:
# process holdings market values
CAD_market_value = 0.0
USD_market_value = 0.0

# last_holdings_row = holdings.iloc[-1]
# print("last_holdings_row", last_holdings_row)

# last_price_row = prices.iloc[-1]
# p = last_price_row[ticker]
# print("last_price_row", last_price_row)

# last rows (by date) from both frames
last_q = holdings.iloc[-1]
last_p = prices.iloc[-1]

# common tickers only (drop any non-ticker columns like CAD/USD if present)
common = last_q.index.intersection(last_p.index)
common = [t for t in common if t not in ('CAD', 'USD')]

# tidy merged frame
last_row = pd.DataFrame({
    'ticker': common,
    'quantity': last_q.loc[common].astype(float).values,
    'price': last_p.loc[common].astype(float).values,
})

print("last_row", last_row)

# sum across rows and add to CAD_market_value and USD_market_value
for index, row in last_row.iterrows():
    currency = ticker_to_currency[row['ticker']]
    if currency == 'CAD':
        CAD_market_value += row['quantity'] * row['price']
    else:
        USD_market_value += row['quantity'] * row['price']

print(f'CAD market value: {CAD_market_value}')
print(f'USD market value: {USD_market_value}')
print(f'total market value in CAD: {CAD_market_value + USD_market_value * LATEST_EXCHANGE_RATE}')

    

last_row        ticker  quantity        price
0        AAPL       0.0   276.043488
1    ACO-X.TO     122.0    54.750000
2         AER      21.0   131.434998
3         AGG       0.0   100.639900
4        AMAT      19.0   232.050003
5        AMSF      45.0    40.404999
6         APO      30.0   130.600006
7        BLBD       0.0    54.584999
8         CEG      25.0   352.625000
9          CG       0.0    52.285000
10     CGY.TO      60.0    46.020000
11  CSH-UN.TO     107.0    20.420000
12       DOLE       0.0    14.050000
13         EA       0.0   201.810394
14     FFH.TO       2.0  2401.320068
15       GOOG      29.0   317.089996
16        GSL     101.0    34.279999
17     HBM.TO       0.0    22.010000
18        IAU      70.0    77.676201
19       ISRG       8.0   569.270020
20       L.TO      80.0    62.180000
21       LUNR     160.0     8.865000
22         MA       0.0   540.679993
23         MP       0.0    58.840000
24      RY.TO      33.0   212.050003
25       SCHP     256.0    26

# Try building from the ground up then compare

In [83]:
# Build holdings from trades
from collections import defaultdict

holdings = defaultdict(int) # ticker: quantity

print(trades_df.head())

for index, row in trades_df.iterrows():
    ticker = row["Ticker"]
    q = row["Quantity"]

    if ticker in holdings:
        print(f'ticker {ticker} bought/sold multiple times, q: {q}')

    # Hard code Loblaw stock split
    if ticker == "L.TO":
        q *= 4

    holdings[ticker] += q
    
print(holdings)


# Build mkt values from holdings and prices

           Ticker Currency  Quantity   Price
Date                                        
2022-05-05     EA      USD      16.0  121.57
2022-05-05   ISRG      USD       8.0  238.04
2022-05-05     MA      USD       5.0  358.41
2022-05-05    TEX      USD      56.0   34.66
2023-04-27   AAPL      USD       4.0  165.39
ticker AGG bought/sold multiple times, q: -59.0
ticker SPY bought/sold multiple times, q: -10.0
ticker XIU.TO bought/sold multiple times, q: -111.0
ticker XBB.TO bought/sold multiple times, q: -557.0
ticker XIU.TO bought/sold multiple times, q: -519.0
ticker AGG bought/sold multiple times, q: -96.0
ticker SPSB bought/sold multiple times, q: -320.0
ticker SPY bought/sold multiple times, q: -25.0
ticker AAPL bought/sold multiple times, q: -4.0
ticker BLBD bought/sold multiple times, q: -61.0
ticker DOLE bought/sold multiple times, q: -108.0
ticker HBM.TO bought/sold multiple times, q: -225.0
ticker CG bought/sold multiple times, q: -51.0
ticker EA bought/sold multiple times, q: 

# compare holdings with our csv holdings

In [84]:
holdings_from_csv = defaultdict(int)

for _, row in last_row.iterrows():
    ticker = row["ticker"]
    q = row["quantity"]

    holdings_from_csv[ticker] += q

print(sorted(holdings_from_csv.items()))
print(sorted(holdings.items()))

assert(sorted(holdings_from_csv.items()) == sorted(holdings.items()))

[('AAPL', 0.0), ('ACO-X.TO', 122.0), ('AER', 21.0), ('AGG', 0.0), ('AMAT', 19.0), ('AMSF', 45.0), ('APO', 30.0), ('BLBD', 0.0), ('CEG', 25.0), ('CG', 0.0), ('CGY.TO', 60.0), ('CSH-UN.TO', 107.0), ('DOLE', 0.0), ('EA', 0.0), ('FFH.TO', 2.0), ('GOOG', 29.0), ('GSL', 101.0), ('HBM.TO', 0.0), ('IAU', 70.0), ('ISRG', 8.0), ('L.TO', 80.0), ('LUNR', 160.0), ('MA', 0.0), ('MP', 0.0), ('RY.TO', 33.0), ('SCHP', 256.0), ('SPSB', 0.0), ('SPY', 0.0), ('TEX', 56.0), ('TLT', 0.0), ('TMUS', 4.0), ('VEEV', 5.0), ('WFG.TO', 23.0), ('WSC', 60.0), ('XBB.TO', 184.0), ('XIU.TO', 0.0)]
[('AAPL', 0.0), ('ACO-X.TO', 122.0), ('AER', 21.0), ('AGG', 0.0), ('AMAT', 19.0), ('AMSF', 45.0), ('APO', 30.0), ('BLBD', 0.0), ('CEG', 25.0), ('CG', 0.0), ('CGY.TO', 60.0), ('CSH-UN.TO', 107.0), ('DOLE', 0.0), ('EA', 0.0), ('FFH.TO', 2.0), ('GOOG', 29.0), ('GSL', 101.0), ('HBM.TO', 0.0), ('IAU', 70.0), ('ISRG', 8.0), ('L.TO', 80.0), ('LUNR', 160.0), ('MA', 0.0), ('MP', 0.0), ('RY.TO', 33.0), ('SCHP', 256.0), ('SPSB', 0.0), ('

### Holding quantites look good so mkt values should be good since we would use the same price data to test